In [1]:
from IPython.display import clear_output
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import requests
import math
import time
from random import randint, random, randrange
from multiprocessing import Pool
from tqdm import tqdm, trange
%matplotlib widget

In [2]:
LR = 1e-2
EPOCHS = 10000
DATASET_LINKS = ["https://s3.dualstack.us-east-1.amazonaws.com/opencitymodel/2019-jun/json/Massachusetts/25021/Massachusetts-25021-000.json",
                "https://s3.dualstack.us-east-1.amazonaws.com/opencitymodel/2019-jun/json/Massachusetts/25021/Massachusetts-25021-001.json",
                "https://s3.dualstack.us-east-1.amazonaws.com/opencitymodel/2019-jun/json/Massachusetts/25021/Massachusetts-25021-002.json",
                "https://s3.dualstack.us-east-1.amazonaws.com/opencitymodel/2019-jun/json/Massachusetts/25021/Massachusetts-25021-003.json",
                "https://s3.dualstack.us-east-1.amazonaws.com/opencitymodel/2019-jun/json/Massachusetts/25021/Massachusetts-25021-004.json",
                "https://s3.dualstack.us-east-1.amazonaws.com/opencitymodel/2019-jun/json/Massachusetts/25021/Massachusetts-25021-005.json",
                "https://s3.dualstack.us-east-1.amazonaws.com/opencitymodel/2019-jun/json/Massachusetts/25021/Massachusetts-25021-006.json"]

In [3]:
def download_dataset(link):
    res = requests.get(link)
    return res.json()
def parse_datasets(raw_datasets):
    parsed_dataset = []
    for i, raw_dataset in enumerate(raw_datasets):
        print(f"Parsing dataset {i + 1}...")
        time.sleep(0.5)
        for building in tqdm(raw_dataset["CityObjects"]):
            parsed_dataset.append((raw_dataset["CityObjects"][building]["attributes"]["latitude"], 
                                   raw_dataset["CityObjects"][building]["attributes"]["longitude"]))
    return zip(*parsed_dataset)


In [4]:
raw_datasets = []
print("Downloading datasets...")
time.sleep(0.5)
for link in tqdm(DATASET_LINKS):
    raw_datasets.append(download_dataset(link))

100%|██████████| 7/7 [00:23<00:00,  3.37s/it]


In [5]:
x_data, y_data = parse_datasets(raw_datasets)
x_data = list(x_data)
y_data = list(y_data)

Parsing dataset 1...


100%|██████████| 40000/40000 [00:00<00:00, 381350.63it/s]


Parsing dataset 2...


100%|██████████| 40000/40000 [00:00<00:00, 362491.97it/s]


Parsing dataset 3...


...

100%|██████████| 40000/40000 [00:00<00:00, 425009.72it/s]


Parsing dataset 6...


100%|██████████| 40000/40000 [00:00<00:00, 451218.76it/s]


Parsing dataset 7...


100%|██████████| 7263/7263 [00:00<00:00, 551740.17it/s]


In [6]:
print(len(x_data))

247263


In [19]:
class Model():
    def __init__(self, x_data, y_data):
        self.x_var = random() * (max(x_data) - min(x_data)) + min(x_data)
        self.y_var = random() * (max(y_data) - min(y_data)) + min(y_data)
        self.loss = 0
        self.results = []
    def get_loss(self, x_data, y_data):
        self.loss = 0
        self.results = []
        for i in range(len(x_data)):
            tmp_loss = math.sqrt(((self.x_var - x_data[i]) ** 2) + ((self.y_var - y_data[i]) ** 2))
            self.results.append(tmp_loss)
            self.loss += tmp_loss
        self.loss /= len(x_data)
    def backpropogate(self, x_data, y_data):
        x_deriv = 0
        for i, x in enumerate(x_data):
            x_deriv += (self.x_var - x) / self.results[i]
        x_deriv /= len(x_data)
        y_deriv = 0
        for i, y in enumerate(y_data):
            y_deriv += (self.y_var - y) / self.results[i]
        y_deriv /= len(y_data)
        
        self.x_var -= LR * x_deriv
        self.y_var -= LR * y_deriv
    def train(self, x_data, y_data):
#         pbar = trange(EPOCHS, desc="loss = {loss:.20f}".format(loss=model.loss))
        fig = plt.figure()
        ax = fig.add_subplot(111)
        plt.ion()
        ax.plot(x_data, y_data, '.', markersize = 0.025, color="black")
        ax.plot(self.x_var, self.y_var, '.', markersize=10, color="red")

        fig.show()

        fig.canvas.draw()
#         return
        for i in range(EPOCHS):
#             clear_output(wait=True)
#             pbar.set_description("loss = {loss:.20f}".format(loss=model.loss))
#             pbar.refresh()
            
            ax.clear()
            ax.plot(x_data, y_data, '.', markersize = 0.025, color="black")
            ax.plot(self.x_var, self.y_var, '.', markersize=10, color="red")
            fig.canvas.draw()
#             self.get_loss(x_data, y_data)
#             self.backpropogate(x_data, y_data)
model = Model(x_data, y_data)   

In [ ]:
model.train(x_data, y_data)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
fig = plt.figure()
buildings = fig.add_subplot(111)
power_plant = fig.add_subplot(111)
plt.ion()
buildings.plot(x_data, y_data, '.', markersize = 0.025, color="black")
power_plant.plot(model.x_var, model.y_var, '.', markersize=10, color="red")

fig.show()
fig.canvas.draw()